# Exploring Some Complex System Dynamics

We want to understand and draw actionable conclusions about some of these systems. Moreover, we want to do so 
* beforehand (i.e., knowledge about an impending financial crisis is better than observations while it's happening)
* in an actionable way (a plan to avoid or mitigate the crisis is more useful than a mere forecast of its occurrence)

In many cases, we observe phase-changes or other specific outcomes in similar systems, and we want to inspect the conditions under which we can avoid (or encourage) these outcomes in our own systems.

## Generative models and simulations

A key mechanism for investigating complex dynamics is a __generative model__.

Generative models are mathematical models, typically implemented via computation, which can yield information about some aspects of a system. So they are, in a way, machine learning models. But they are different from the most common, predictive ML models, that look at a set of outcomes (data) and try to predict future results (next quarter's sales) or more general results (transcribing speech).

> Generative models posit some __data generating process__ which may be similar to the dynamics in our systems.

The models -- also called simulations -- can then produce lots of outcomes based on some initial parameters and the data generating process. 

By looking at the process, the parameters, and the outcomes, we can learn about the behavior of such a system as parameters are changed.

We can then become aware of risks or opportunities that might otherwise be hidden.

## Some key systems models

Today, we'll look at a few classes of models that can reveal some of the challenging dynamics of complexity.

We'll take a look at ...
* network models
* automata

Next time we'll look at
* agent-based models
* path dependence

along with examples where they might apply.

## Network models

Many processes exhibit the consequences of network effects. These include both natural and social systems, including business systems.

We may want to exploit network effects to
* generate sales
* dominate/control a product category
* spread positive sentiment around our product or firm

Conversely we may want to avoid or disrupt network effects to
* lower systemic risks ("domino effect" cascading failures)
* protect intellectual property (limiting the spread of illicit use)
* protect secrets and first-mover advantage

But not every network exhibits the massive spread we may want to encourage or avoid.

__We can experiment on synthetic networks and learn critical dynamics__

### Erdos-Renyi Graphs

We can think of our scenario of interest as a graph:
* a set of nodes, which might represents individuals or firms,
* and a set edges connecting pairs of nodes, which might represent communications, transactions, or business relationships.

The Erdos-Renyi model (Paul Erdős, Alfréd Rényi, Edgar Gilbert, separate work ~1959) considers a family of graphs that contain some fixed number of nodes __n__ and some fixed probability __p__ that any two nodes are connected.

This is a simplistic model, but it produces interesting behavior.

If __p__ is close to zero, we can easily imagine that the graph is unlikely to be __connected__ or provide some route between all pairs of nodes.

On the other hand, if __p__ is close to one, it is not surprising that the graph ends up connected.

In between, things get interesting. Let's take a quick look at how the probability of an E-R graph being connected varies as __p__ changes.

How do we do this?

The easiest way to explore this -- and a mechanism I recommend because it works even when other methods are intractable or deeply complicated -- is to simulate the system and count outcomes.

In [ ]:
import networkx as nx
import numpy as np

def make_er_graph(n, p):
    G = nx.Graph()
    nodes = range(n)
    G.add_nodes_from(nodes)
    G.add_edges_from( (i, j) for i in nodes for j in nodes if  i > j and np.random.random() < p )
    return G

g = make_er_graph(20, 0.1)
nx.draw(g)

In [ ]:
nx.is_connected(g)

In [ ]:
g1 = make_er_graph(20, 0.5)
nx.draw(g1)

In [ ]:
nx.is_connected(g1)

In [ ]:
def test_connectivity_for_random_graph(n, p):
    return nx.is_connected(make_er_graph(n, p))

def prob_connected(n, p, samples):
    return sum( (test_connectivity_for_random_graph(n, p) for i in range(samples)) ) / samples

prob_connected(20, 0.2, 100)

In [ ]:
from matplotlib import pyplot as plt

n = 8
samples = 200
edge_probs = np.linspace(0, 1, 20)
connectivity_probs = [prob_connected(n, p, samples) for p in edge_probs]

plt.plot(edge_probs, connectivity_probs)

One interesting phenomenon is that this transition point becomes more sudden as the graph grows

In [ ]:
n = 20
connectivity_probs = [prob_connected(n, p, samples) for p in edge_probs]

plt.plot(edge_probs, connectivity_probs)

In [ ]:
n = 40
connectivity_probs = [prob_connected(n, p, samples) for p in edge_probs]

plt.plot(edge_probs, connectivity_probs)

Erdos and Renyi discovered that the critical value, at which the connectivity probability quickly transitions from 0 to 1, is $\frac{log(n)}{n}$

That number gets close to zero for large n.

In [ ]:
np.log(100)/100

### Takeaway

What are some takeaways from this experiment?

For large graphs with random connections, even a tiny probability of connection will likely connect the whole graph.

* This could be a good thing -- if you're spreading the word about your new product, or signing up folks to transact on your new money platform.

* But it could also be a terrible thing if the "message" being passed is a new virus or a pro-genocide meme.

In our experiment, we looked at the effect of connectivity. But we can just as easily fix a connectivity probability and ask about the effect of scaling the graph. 

Because $\frac{log(n)}{n}$ gets small as n gets big, we can say that for *any* connectivity probability (above zero), there will be a graph size large enough that it is ~100% likely to be connected.

In plain language: we've discovered the math behind the assertion that adding more people (or more anything) to a graph makes it inevitable that anything and everything can spread everywhere.

If this is the spread of ...
* a lifesaving product or knowledge, we have reason to rejoice
* our tech platform product, we had better be careful about unintended consequences of that product, because they can be everywhere
* financial instability due to propagated risk (as in 2008), we can see that failure was inevitable with scale

__How does this connect to the distributions we talked about earlier?__

Highly connected networks mean that when signals (info, memes, viruses, etc.) spread, the transmission is multiplicative (as we've seen with Covid and $R_0$) and, of course, the events are not independent -- they are linked by the relations in the graph. So spread through a network can lead to power-law distributions. Depending on the exponent, these may have fat tails and hide a large number of dramatic surprises that would not be expected from thin-tailed distributions.

A great overview of network effects in financial risk is https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3651864

#### More is Different

This phenomenon -- that "more is different" or quantity can become quality -- is the idea behind, and title of, a 1972 paper by Philip Anderson often regarded as inaugurating the study of complex systems: https://science.sciencemag.org/content/177/4047/393

Since humans and human institutions are used to linear change, this discovery of mechanisms behind sudden, non-linear change is a key tool in designing the outcomes we want in the world.

## Automata and agent-based models

Another kind of generative or simulation-based model which can offer insights into the dynamics of complexity is the agent-based model.

* An agent-based model is just a simulation of a number of agents (a bit like imaginary characters) who act according to some rule in an environment that also features some rule.

* Agent-based models are associated with automata because they are often implemented in simplified "world" akin to those of classic cellular automata like Conway's Game of Life.

Let's take a quick look: https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life

The Game of Life is fascinating for numerous reasons, but the one we're focusing on today is another key phenomenon of complex systems, __emergence__.

__Emergence__ is another way of describing the manifestation of interesting, potentially surprising, nonlinear outcomes from a simple set of mechanistic rules.

Game of Life has a trivial ruleset and a passive environment and gives rise to astonishing complexity.

### Using agent-based models

The rules of "life" are interesting but may not be related to any particular real-world system.

We can borrow the automaton idea -- and substitute rules that we would like to investigate -- to see if our rules lead to favorable outcomes. With an experimental platform like this, we can then adjust our rules in the hope of creating different outcomes.

### Schelling Segregation Model

To get a feel for how this approach might be informative, we'll look at one of the earliest and most famous agent-based models: the __Schelling Segregation Model__

Allen Downey summarizes this model/thought experiment brilliantly in his book *Think Complexity*:

> The Schelling model of the world is a grid where each cell represents a house. The houses are occupied by two kinds of agents, labeled red and blue, in roughly equal numbers. About 10% of the houses are empty.
>
> At any point in time, an agent might be happy or unhappy, depending on the other agents in the neighborhood, where the “neighborhood" of each house is the set of eight adjacent cells. In one version of the model, agents are happy if they have at least two neighbors like themselves, and unhappy if they have one or zero.
>
> The simulation proceeds by choosing an agent at random and checking to see whether they are happy. If so, nothing happens; if not, the agent chooses one of the unoccupied cells at random and moves.
>
> You will not be surprised to hear that this model leads to some segregation, but you might be surprised by the degree. From a random starting point, clusters of similar agents form almost immediately. The clusters grow and coalesce over time until there are a small number of large clusters and most agents live in homogeneous neighborhoods.
>
> If you did not know the process and only saw the result, you might assume that the agents were racist, but in fact all of them would be perfectly happy in a mixed neighborhood.

Let's implement this model to see
* what a simple agent-based model implementation looks like
* how the "homophily index," or fraction of similar neighbors required for happiness, affects the overall segregation of the grid

In [ ]:
size = 15
homophily_index = 0.3

In [ ]:
def make_grid(size):
    grid = np.random.uniform(0, 1, (size,size))
    grid[grid>0.55] = 1
    grid[grid<0.45] = 2
    grid[grid<1]=0
    return grid

grid = make_grid(size)
grid

In [ ]:
plt.magma()
plt.imshow(grid)
plt.colorbar()

In [ ]:
np.argwhere(grid==0)

In [ ]:
import random

def pick_random_agent(grid):
    agent_locations = np.argwhere(grid != 0)
    loc_index = random.randint(0, agent_locations.shape[0]-1)
    return (agent_locations[loc_index][0], agent_locations[loc_index][1])

def pick_empty_loc(grid):
    empty_locations = np.argwhere(grid == 0)
    loc_index = random.randint(0, empty_locations.shape[0]-1)
    return (empty_locations[loc_index][0], empty_locations[loc_index][1])

In [ ]:
agent = pick_random_agent(grid)
agent

In [ ]:
agent_group = grid[agent[0], agent[1]]
agent_group

In [ ]:
neighborhood = grid[agent[0]-1:agent[0]+2, agent[1]-1:agent[1]+2]
neighborhood

In [ ]:
similar_neighbors_locs = (neighborhood == agent_group)
similar_neighbors_locs

In [ ]:
similar_neighbors = similar_neighbors_locs.sum() - 1
similar_neighbors

In [ ]:
def do_update(grid):
    agent = pick_random_agent(grid)
    agent_group = grid[agent[0], agent[1]]
    neighborhood = grid[agent[0]-1:agent[0]+2, agent[1]-1:agent[1]+2]
    similar_neighbors = (neighborhood == agent_group).sum() - 1
    is_happy = (similar_neighbors / 8) > homophily_index
    if not is_happy:
        new_loc = pick_empty_loc(grid)
        grid[agent[0], agent[1]] = 0
        grid[new_loc[0], new_loc[1]] = agent_group

In [ ]:
plt.imshow(grid)

In [ ]:
for i in range(10 * size**2):
    do_update(grid)
    
plt.imshow(grid)

In [ ]:
size = 100

grid = make_grid(size)
plt.imshow(grid)

In [ ]:
for i in range(2 * size**2):
    do_update(grid)
    
plt.imshow(grid)

In [ ]:
for i in range(2 * size**2):
    do_update(grid)
    
plt.imshow(grid)

In [ ]:
for i in range(2 * size**2):
    do_update(grid)
    
plt.imshow(grid)

In [ ]:
homophily_index = 0.4

size = 100

grid = make_grid(size)
plt.imshow(grid)

In [ ]:
for i in range(4 * size**2):
    do_update(grid)
    
plt.imshow(grid)

It would be interesting to plot the homophily index vs. the number of iterations before a particular segregation level is met, but that is a bit beyond what we have time for today.

### Takeaway

What are some takeaways from this experiment?

We can test hypotheses which may be critical to real-world phenomena within a highly artificial "small world" and still learn critical insights.

* For example, we might want to test the following hypothesis:
    * "Modest homophily values like 30% are insufficient to generate segregation -- something else is necessary."
    * *We can see that the hypothesis is clearly false.*

Of course, the model cannot tell you how to manage your society, business, or project. But it can provide indicators you can use when designing for target outcomes.

#### How widely applicable are these agent-based models?

Thomas Schelling, an economist with work in other disciplines as well, published his model in 1969. 

Since then, numerous researchers have used ABMs to explore a wide range of phenomena.

For example, Joshua Epstein has used ABMs to explore how simple rules can explain diverse phenomena in the realm of social contagion including how
* a jury can unanimously vote to convict when only a minority of participants believe the defendant is guilty
* diversity in "trigger points" can make a mob more likely to turn violent
* soldiers can become susceptible to committing mass killings and other atrocities

If this is interesting to you, check out his book, *Agent_Zero*: https://press.princeton.edu/books/hardcover/9780691158884/agentzero

Long in use in the physical sciences, simulation-based modeling has become a lot more popular recently, as larger amounts of compute power and easy-to-use, performant tools (like SciPy/PyData) have become available.

For example, ABMs are illuminating areas of macroeconomics where the neoclassical *homo economicus* model has broken down.

#### How does this connect to the distributions we talked about earlier?

Although it is convenient to formulate and render cellular automata like these in a "grid world," they can actually be interpreted as graphical models, so they are not as far away from networks as they might appear at first. And, as areas of the "grid" are assimilated to one factor or another, there are multiplicative effects: for example, in this model, the colored areas (~ 2-dimensional) become overwhelmingly larger than the frontiers (~ 1-dimensional).

The homophily model we've looked at here might just as easily describe users of one or another mobile phone or communications platform (in non-geographical cases, the dimensions may represent other aspects of a social or product space) -- so there are definitely applications in business.

## Exploring path dependence

Simple models may evaluate a distribution of outcomes for an individual, team, firm, or other group over a series of choices.

For example, when choosing what product to prioritize for the next quarter, projections might assign probabilities and expected profits to different market scenarios and product choices. A business unit might choose to focus on the product with highest expected profit across the projected business scenarios.

However -- as anyone familiar with the consequences of technical debt can tell you -- your next choice is rarely made with a blank-slate starting point. We all have to live with the consequences of our previous choices, and that can change the expected outcome dramatically. 

* In other words, our outcomes are not purely dependent on a current decision. They are dependent on the path of prior steps in the outcome space.

### Simple investment model

We'll take a look at a simple investment (or gambling) model which produces reliable positive returns when viewed from the an average (or expectation) perspective, but yields ruinous losses when viewed from the path-dependent perspective of any actual investor (or gambler)

__The business proposition__

* 50/50 risk of success or failure
* Success returns 50 cents on the dollar (i.e., \\$1 invested returns \\$1.50)
* Failure produces a loss of 40 cents (i.e., in the failure scenario, one recoups \\$0.60 from each \\$1 invested)

Traditional expectation:

In [ ]:
0.5 * 1.5 + 0.5 * 0.60

We can simulate that to get a better idea of the deviation from the ideal average

In [ ]:
sample_size = range(100, 10000, 100)

outcomes = []

for i in sample_size:
    draws = np.random.uniform(0, 1, (i))
    draws[draws > 0.5] = 1.50
    draws[draws < 1] = 0.6
    outcomes.append(draws.mean())
    
plt.plot(sample_size, outcomes)

So it looks like, even for small samples or "bad luck" we should do pretty well with this sort of investment.

__Ensemble average vs. time average__

But this form of average assumes that we start in the same position prior to each investment or bet.

* It's a bit like looking at hundreds or thousands of individuals or firms each making one bet. On average, they will (collectively) do well!

But let's change our perspective for a moment and look at one individual or firm making a sequence of small bets/investments.

* If they make $2n$ investments, we would expect about $n$ to yield the \\$1.50 and the other $n$ to yield the \\$0.60
* So the end result would be $(1.5)^n*(0.6)^n = [(1.5)(0.6)]^n = 0.9^n$

Wait ... $0.9^n$ doesn't look very good. In fact, it will go very quickly to zero for any significant $n$

Just to be sure, let's simulate this as well:

In [ ]:
steps=200
simulations=10000
draws = np.random.uniform(0, 1, (simulations, steps))
draws[draws > 0.5] = 1.50
draws[draws < 1] = 0.6
outcomes = draws.prod(axis=1)
plt.hist(outcomes, bins=100)

Just for comparison, our expected value after `steps` investments

In [ ]:
expected = 1.05 ** steps
expected

In [ ]:
outcomes[outcomes < 0.1].size / simulations

In [ ]:
outcomes[outcomes < 1].size / simulations

In [ ]:
outcomes[outcomes > 2].size / simulations

In [ ]:
outcomes[outcomes >= expected].size / simulations

__A dramatic view of the "lifelines" of a number of agents facing a similar set of options__

<img src='images/ergo.webp' width=700>

From: https://www.nature.com/articles/s41567-019-0732-0

#### Takeaway

When does this occur in real life?

Although our specific numbers in the present example are contrived, path dependence is a critical factor in many real-world systems:
* economic actors
* health outcomes
* hiring and promotion
* education
* criminal justice
* participation in risk-taking and investment activities

__How does this connect to the distributions and patterns we've been talking about?__

Notice that, in the path-dependent case,
* we have a *series of multiplied values which are not independent*
    * (since each multiplication is  dependent on prior state) 
* where, in the ensemble expectation, we *assumed* that all of the events (values being multiplied) are independent
    * (they only depend on the "rules of the game" -- every trial starts with 1 dollar)
    
Once again, we see a compounding effect leading to drastically large (or small) numbers. 

A concrete example is insurance pools. A sufficiently large and diverse business can "self insure" anything from employee health costs to its own fleet of vehicles. Such self insurance can work, provided the losses are independent enough that the ensemble average holds.

If a company's employees were all concentrated in an area with common health hazards (say, contaminated air or ground water) then the sequence of repeated of heath-cost losses would not be independent -- risk would be magnified as health losses compound over time.

__How do we use this knowledge?__

Any time we are looking to achieve an "average" result over time, we can ask whether the steps are truly independent. As a technology example, we may have a device that we deploy in the field which features high uptime (time between failures). 
* To achieve long-term reliability, we want to ensure that the device is as stateless as possible when it recovers
* If a device retains state (e.g., internal storage or config) which affect its future success (after recovering from a failure) then the sequence of failures becomes path dependent

## Journey Checkpoint

We've now taken a brief look at three ways of thinking about interconnected, complex systems.

* Specifically, we've learned how to recognize system that can hide huge risk or volatility.

This motivates a natural question:

* What can we do to mitigate these risks and improve our chances of success?

We can now emerge from our detailed investigation with the insight to produce possible answers.